## 第一部分：了解 nn.Module的基本操作

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [17]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [19]:
for name, param in model.named_parameters():
    print('aaa:',name, 'bbb:',param.shape)

aaa: conv1.weight bbb: torch.Size([64, 3, 7, 7])
aaa: bn1.weight bbb: torch.Size([64])
aaa: bn1.bias bbb: torch.Size([64])
aaa: layer1.0.conv1.weight bbb: torch.Size([64, 64, 3, 3])
aaa: layer1.0.bn1.weight bbb: torch.Size([64])
aaa: layer1.0.bn1.bias bbb: torch.Size([64])
aaa: layer1.0.conv2.weight bbb: torch.Size([64, 64, 3, 3])
aaa: layer1.0.bn2.weight bbb: torch.Size([64])
aaa: layer1.0.bn2.bias bbb: torch.Size([64])
aaa: layer1.1.conv1.weight bbb: torch.Size([64, 64, 3, 3])
aaa: layer1.1.bn1.weight bbb: torch.Size([64])
aaa: layer1.1.bn1.bias bbb: torch.Size([64])
aaa: layer1.1.conv2.weight bbb: torch.Size([64, 64, 3, 3])
aaa: layer1.1.bn2.weight bbb: torch.Size([64])
aaa: layer1.1.bn2.bias bbb: torch.Size([64])
aaa: layer2.0.conv1.weight bbb: torch.Size([128, 64, 3, 3])
aaa: layer2.0.bn1.weight bbb: torch.Size([128])
aaa: layer2.0.bn1.bias bbb: torch.Size([128])
aaa: layer2.0.conv2.weight bbb: torch.Size([128, 128, 3, 3])
aaa: layer2.0.bn2.weight bbb: torch.Size([128])
aaa: layer

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [20]:
input_ = torch.randn(1, 3, 128, 128)
input_

tensor([[[[ 0.5852,  1.0942, -0.8445,  ..., -0.5095, -0.0897,  2.0410],
          [ 0.0598,  0.5741,  1.3847,  ...,  0.7231,  1.0707, -0.1507],
          [ 1.7219,  0.7560,  0.2560,  ..., -1.2163, -0.0154, -0.7877],
          ...,
          [-0.6013,  0.1469, -0.4294,  ..., -0.2511, -0.6213,  0.4176],
          [-0.7369,  0.8902,  0.0079,  ..., -1.2185, -0.7383,  0.3169],
          [ 0.2427, -1.5843, -1.3168,  ...,  0.1003,  1.0662,  0.2393]],

         [[-0.8259, -0.7201,  0.0575,  ..., -1.4817,  1.3714, -0.2648],
          [ 1.7483, -0.0338,  0.5948,  ..., -0.0932, -0.3720, -0.3951],
          [-0.2381,  0.4801, -0.2078,  ...,  1.3701, -0.8560, -1.2664],
          ...,
          [-1.1189,  0.2391,  0.7367,  ...,  0.3612,  0.1360,  0.3627],
          [ 0.5191, -0.9273,  1.5703,  ..., -0.2617, -0.7294,  0.8638],
          [ 0.5738, -0.3648,  0.1828,  ..., -1.1349,  0.6002, -0.0592]],

         [[-0.1387, -0.9102,  0.4443,  ..., -0.8908,  0.5625, -0.5389],
          [-0.7360,  0.4541, -

In [24]:
for param in model.parameters():
    param.requires_grad = False

In [11]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [25]:
input_ = torch.randn(10,12)
## 示範
Linear = nn.Linear(12,10)
BN = nn.BatchNorm1d(10)

In [26]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.sequential = nn.Sequential(nn.Linear(12,10), nn.BatchNorm1d(10))
        self.repeat_linear = nn.ModuleList([nn.Sequential(nn.Linear(10,10), nn.BatchNorm1d(10)) for _ in range(10)])
        self.output = nn.Linear(10, 3)

    def forward(self, x):
        x = self.sequential(x)
        for module in self.repeat_linear:
            x = module(x)
        x = self.output(x)
        return x

In [27]:
model = Model()

In [28]:
model

Model(
  (sequential): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (repeat_linear): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Sequential(
      (0): Linear(in_features=10, out_fe

In [19]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [20]:
outupt

tensor([[-0.6768, -0.0901, -0.0031],
        [ 0.4180, -0.4599,  1.8801],
        [-0.1755, -0.1612, -0.0391],
        [-0.0480,  0.0881, -0.6903],
        [-1.3550,  0.7311, -1.3749],
        [ 0.2309, -0.4470, -0.0557],
        [-0.2988, -0.1650, -0.1030],
        [-0.5010, -0.6347, -0.5067],
        [ 0.6583, -0.6786,  0.5469],
        [ 0.3372, -0.7615,  0.8639]], grad_fn=<AddmmBackward>)